### Run the (SageMaker or local) detector for all images in a folder


In [ ]:

import os, json, boto3, base64, requests
from PIL import Image
from tqdm import tqdm

IMAGE_DIRECTORY = "/home/gfuhr/projects/data/immediate_action/people_v1/"
LOCAL_MODEL = True
if LOCAL_MODEL:
    LOCAL_ENDPOINT = "http://localhost:8080/invocations"
else:
    ENDPOINT_NAME = "sagemaker-inference-server-endpoint"
    AWS_REGION = "eu-west-1"
    sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=AWS_REGION)
MODEL_TYPE = "object_detection"



def encode_image_base64(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode()

def make_aws_sagemaker_request(image_b64, model_type):
    resp = sagemaker_runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/json",
        Body=json.dumps({"image_base64": image_b64, "model": model_type}),
    )
    return json.loads(resp["Body"].read().decode())

def make_local_request(image_b64, model_type, model_endpoint="http://localhost:8080/detect/"):
    resp = requests.post(model_endpoint, json={
        "image_base64": image_b64,
        "model": model_type
    })
    if resp.status_code != 200:
        print("Failed for image")
        return {}
    return resp.json()

images = sorted([f for f in os.listdir(IMAGE_DIRECTORY) if f.endswith('.png')])
coco = {
    "images": [],
    "annotations": [],
    "categories": [{"id": 0, "name": "person"}]
}
ann_id, img_id = 1, 1

for fname in tqdm(images):
    path = os.path.join(IMAGE_DIRECTORY, fname)
    img = Image.open(path)
    w, h = img.size
    img_b64 = encode_image_base64(path)
    if LOCAL_MODEL:
        detections = make_local_request(img_b64, MODEL_TYPE)
    else:
        detections = make_aws_sagemaker_request(img_b64, MODEL_TYPE)
    coco["images"].append({"id": img_id, "file_name": fname, "width": w, "height": h})
    for det in detections["detections"]:
        if det["label"] != 0:
            continue
        x1, y1 = det["bbox"]["top_left"]["x"], det["bbox"]["top_left"]["y"]
        x2, y2 = det["bbox"]["bottom_right"]["x"], det["bbox"]["bottom_right"]["y"]
        coco["annotations"].append({
            "id": ann_id,
            "image_id": img_id,
            "category_id": 0,
            "bbox": [x1, y1, x2 - x1, y2 - y1],
            "score": det.get("score", 1.0),
        })
        ann_id += 1
    img_id += 1

with open(os.path.join(IMAGE_DIRECTORY, "annotations.json"), "w") as f:
    json.dump(coco, f)


In [ ]:
import fiftyone as fo
import os

dataset_name = "people_v1"
coco_json_path = os.path.join(IMAGE_DIRECTORY, "annotations.json")
images_dir = IMAGE_DIRECTORY

if dataset_name in fo.list_datasets():
    fo.delete_dataset(dataset_name)

dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=images_dir,
    labels_path=coco_json_path,
    name=dataset_name,
)